In [1]:
# setting
import pandas as pd
from pandasql import sqldf
data = pd.read_csv('./data.csv')

#### 특정 컬럼의 통계량 확인

In [2]:
# pandas
data['release_year'].describe()

count    8807.000000
mean     2014.180198
std         8.819312
min      1925.000000
25%      2013.000000
50%      2017.000000
75%      2019.000000
max      2021.000000
Name: release_year, dtype: float64

In [3]:
query = \
'''
WITH temp_s AS (SELECT COUNT(release_year) as count,
                       AVG(release_year) as mean,
                       MIN(release_year) as min,
                       MAX(release_year) as max
                FROM data),
     temp_q AS (SELECT PERCENT_RANK() OVER (ORDER BY release_year) AS rank,
                       release_year
                FROM data)

SELECT
    temp_s.count as count,
    temp_s.mean as mean,
    SUM((temp_q.release_year - (temp_s.mean)) * (temp_q.release_year - (temp_s.mean))) / temp_s.count as var,
    temp_s.min as min,
    ROUND(AVG(CASE WHEN ROUND(temp_q.rank, 1) = 0.2 THEN temp_q.release_year END), 0) AS '25%',
    ROUND(AVG(CASE WHEN ROUND(temp_q.rank, 1) = 0.5 THEN temp_q.release_year END), 0) AS '50%',
    ROUND(AVG(CASE WHEN ROUND(temp_q.rank, 1) = 0.7 THEN temp_q.release_year END), 0) AS '75%',
    temp_s.max as max
FROM temp_s, temp_q
'''
sqldf(query) # sqldf가 SQRT()를 지원하지 읺이 std를 var로 대체

,count,mean,var,min,25%,50%,75%,max
0,8807,2014.180198,77.771435,1925,2012.0,2017.0,2019.0,2021
